# 先从产品以及相关的订单销售数据开始

In [1]:
from imports import *
from utils import *

In [2]:
order_m_path = os.path.join(BASE_DIR, 'order_monthly_template.xlsx')
order_m = pd.read_excel(order_m_path)

In [3]:
order_m

,product_id,month,count_raw
0,1,2018-01-01,NaN
1,1,2018-02-01,NaN
2,1,2018-03-01,NaN
3,1,2018-04-01,NaN
4,1,2018-05-01,NaN
...,...,...,...
139,4,2022-08-01,NaN
140,4,2022-09-01,NaN
141,4,2022-10-01,NaN
142,4,2022-11-01,NaN


In [4]:
order_m['year'] = order_m['month'].apply(lambda x: x.year)

In [5]:
# 生成数据

In [6]:
# from yearly data to monthly breakdown

# grow = 100
# total_year = intercept * 12 + grow * 12/2
# total_year

In [7]:
# 读取年度数据
df = pd.read_excel(BASE_INFO_PATH, sheet_name= '产品及销量')

In [8]:
df

,产品,2018,2019,2020,2021,2022
0,A,2000.0,1500.0,1500.0,1.500000e+03,1.500000e+03
1,B,NaN,2000.0,2000.0,2.000000e+03,2.000000e+03
2,C,NaN,NaN,NaN,3.000000e+03,3.000000e+03
3,D,NaN,NaN,NaN,NaN,4.500000e+03
4,销量,2018.0,2019.0,2020.0,2.021000e+03,2.022000e+03
5,A,20000.0,40000.0,80000.0,1.000000e+05,1.200000e+05
6,B,NaN,70000.0,100000.0,1.500000e+05,2.300000e+05
7,C,NaN,NaN,NaN,1.500000e+04,5.000000e+04
8,D,NaN,NaN,NaN,NaN,3.500000e+04
9,年度总销量,20000.0,110000.0,180000.0,2.650000e+05,4.350000e+05


In [9]:
df_price = df.loc[5:8]

df_price


,产品,2018,2019,2020,2021,2022
5,A,20000.0,40000.0,80000.0,100000.0,120000.0
6,B,NaN,70000.0,100000.0,150000.0,230000.0
7,C,NaN,NaN,NaN,15000.0,50000.0
8,D,NaN,NaN,NaN,NaN,35000.0


In [17]:
df_price_adj = df_price.melt(id_vars=['产品'], var_name='year', value_name='count').dropna()
df_price_adj = pd.merge(df_price_adj,get_product()[['product_id','product_code']], left_on = '产品', right_on='product_code', how='left')
df_price_adj

,产品,year,count,product_id,product_code
0,A,2018,20000.0,1,A
1,A,2019,40000.0,1,A
2,B,2019,70000.0,2,B
3,A,2020,80000.0,1,A
4,B,2020,100000.0,2,B
5,A,2021,100000.0,1,A
6,B,2021,150000.0,2,B
7,C,2021,15000.0,3,C
8,A,2022,120000.0,1,A
9,B,2022,230000.0,2,B


In [28]:
'product_id == {} and year = {}'.format(1, 2018)

'product_id == 1 and year = 2018'

In [18]:
# p = df_price.iloc[0,:].values[1:].reshape(-1,1)
# _id = np.array([i for i in range(1, len(p)+1)]).reshape(-1,1)
# intercept, slope = linear_regression_one_variable(_id,p)

In [43]:
# 

def gen_rand(l, total_year,intercept = None, growth = 'linear'):
    """生成随机月度销量数据"""
    
    # total_year = 20000


    if intercept is None:
        intercept = int(total_year * 0.05)

    # exponential growth
    if growth == 'exponential':
        yearly_growth = 0.4
        growth_rate_monthly = np.power(1 + yearly_growth, 1/12) - 1
    # linear growth
    if growth == 'linear':
        yearly_base = intercept * 12
        yearly_rest= total_year - yearly_base
        year_end_growth = yearly_rest * 2 / 12
        growth_count = year_end_growth / 12
        

    
    var = 0.2

    
    g = []
    
    # TODO P3: redo this section
    for i in range(l):
        if i == 0:
            base_line = intercept
        else:
            # base_line = intercept + growth_rate_monthly
            if growth == 'exponential':
                base_line = base_line * (1 + growth_rate_monthly)
            if growth == 'linear':
                base_line = intercept + growth_count * i
            
        if growth == 'exponential':
            _ = base_line * (1+np.random.normal(loc = 0, scale = var))
        
        if growth == 'linear':
            _ = base_line * (1+np.random.normal(loc = 0, scale = var))
            
        _ = np.round(_)
        _ = int(_)
        
        # check non-negative
        if _<0:
            _ = 0
            
        g.append(_)
        
    return g

df_g_list = []


for product in order_m['product_id'].unique():
    # product
    df_this_prod = order_m[order_m['product_id'] == product]
    
    last_month_of_this_product = 0
    
    for idx, year in enumerate(df_this_prod['year'].unique()):


        year_total = df_price_adj.query('product_id == {} and year == {}'.format(product,year))['count'].values[0]
        
        
        print(product,year, year_total)
        
        df_this_year = df_this_prod[df_this_prod['year'] == year]
        
        l = df_this_year.shape[0]
        
        if idx > 0:
            # intercept use last year
            intercept = last_month_of_this_product
        else:
            # none means 5% of 1st year
            intercept = None
        
        df_this_year['count_raw'] = gen_rand(l, year_total, intercept = intercept)
        last_month_of_this_product = df_this_year['count_raw'].values[-1]
        
        df_g_list.append(df_this_year)

df_order_g = pd.concat(df_g_list)

1 2018 20000.0
1 2019 40000.0
1 2020 80000.0
1 2021 100000.0
1 2022 120000.0
2 2019 70000.0
2 2020 100000.0
2 2021 150000.0
2 2022 230000.0
3 2021 15000.0
3 2022 50000.0
4 2022 35000.0


/var/folders/5r/tzcqr9wx0n9f9901lytz5c7m0000gn/T/ipykernel_7228/2809845930.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
df_order_g

,product_id,month,count_raw,year
0,1,2018-01-01,1057,2018
1,1,2018-02-01,997,2018
2,1,2018-03-01,1219,2018
3,1,2018-04-01,1029,2018
4,1,2018-05-01,1783,2018
...,...,...,...,...
139,4,2022-08-01,3418,2022
140,4,2022-09-01,3460,2022
141,4,2022-10-01,4003,2022
142,4,2022-11-01,2870,2022


In [45]:
px.line(df_order_g, x = 'month', y = 'count_raw', color = 'product_id')

In [46]:
df_order_g

# group by product_id and month
df_order_g['year'] = df_order_g['month'].apply(lambda x: x.year)

In [50]:
df_order_g

,product_id,month,count_raw,year
0,1,2018-01-01,1057,2018
1,1,2018-02-01,997,2018
2,1,2018-03-01,1219,2018
3,1,2018-04-01,1029,2018
4,1,2018-05-01,1783,2018
...,...,...,...,...
139,4,2022-08-01,3418,2022
140,4,2022-09-01,3460,2022
141,4,2022-10-01,4003,2022
142,4,2022-11-01,2870,2022


In [51]:
# 年度以及月度产量

df_order_g.groupby(['year', 'product_id']).sum()

/var/folders/5r/tzcqr9wx0n9f9901lytz5c7m0000gn/T/ipykernel_7228/3979653090.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



count_raw
year product_id           
2018 1               20265
2019 1               39177
     2               74177
2020 1               79110
     2               99953
2021 1              102481
     2              128347
     3               14334
2022 1              115979
     2              236288
     3               55389
     4               34820

# 保存

In [49]:
df_order_g.to_csv(os.path.join(GE_DIR, 'order_monthly.csv'), index = False)